In [ ]:
import IPython

import random
import scipy
import numpy as np # manipulate N-dimensional arrays
import pandas as pd # data frame
import matplotlib.pyplot as plt # data plotting
import seaborn as sns # advanced data plotting
from sklearn import preprocessing # basic ML models
# import scipy # scientific computing library

import tensorflow as tf
import keras
import sklearn as sk

from collections import Counter # Permet de compter le nombre d'éléments qui possèdent chaque label
from keras.utils import to_categorical # One-hot encoding

from keras.models import Model, Sequential
from keras.layers import Input, Dense, BatchNormalization, Dropout, ReLU, Activation, Flatten, Conv2D, MaxPooling2D

from keras.backend import clear_session
from keras.optimizers import Adam

from keras.callbacks import EarlyStopping

import os
from keras import backend as K

from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold

from tqdm import tqdm #progress bar


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Préparation des données

In [ ]:
data = pd.read_csv("/content/drive/My Drive/stanford-covid-vaccine/data_train.csv")

In [ ]:
 def mcrmse(y_true, y_pred):
        cmse = tf.reduce_mean(tf.square(y_true - y_pred), axis=1)
        res = tf.reduce_mean(tf.sqrt(cmse), axis=1)
        return res

In [ ]:
data.head()

,Unnamed: 0,id,sequence,index_sequence,seq_be,seq_af,structure,struct_be,struct_af,predicted_loop_type,loop_type_be,loop_type_af,bpps_0,bpps_1,bpps_2,bpps_3,bpps_4,bpps_5,bpps_6,bpps_7,bpps_8,bpps_9,bpps_10,bpps_11,bpps_12,bpps_13,bpps_14,bpps_15,bpps_16,bpps_17,bpps_18,bpps_19,bpps_20,bpps_21,bpps_22,bpps_23,bpps_24,bpps_25,bpps_26,bpps_27,...,bpps_74,bpps_75,bpps_76,bpps_77,bpps_78,bpps_79,bpps_80,bpps_81,bpps_82,bpps_83,bpps_84,bpps_85,bpps_86,bpps_87,bpps_88,bpps_89,bpps_90,bpps_91,bpps_92,bpps_93,bpps_94,bpps_95,bpps_96,bpps_97,bpps_98,bpps_99,bpps_100,bpps_101,bpps_102,bpps_103,bpps_104,bpps_105,bpps_106,signal_to_noise,SN_filter,reactivity,deg_Mg_pH10,deg_pH10,deg_Mg_50C,deg_50C
0,0,id_001f94081_0,G,1,O,G,.,O,.,E,O,E,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.006554,0.009221,0.008094,0.021786,0.0,0.0,0.0,0.0,0.0,0.00112,0.0,0.0,0.0,0.0,0.0,0.0,0.004653,0.002415,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.001411,0.0,0.002758,0.005980,0.005979,0.013887,0.0,0.0,0.001528,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.009526,0.0,0.0,0.011146,0.0,0.0,0.011636,0.0,0.0,0.010538,0.0,0.0,0.014674,6.894,1,0.3297,0.7556,2.3375,0.3581,0.6382
1,1,id_001f94081_1,G,2,G,A,.,.,.,E,E,E,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.018205,0.005112,0.038653,0.000000,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.004323,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.004437,0.0,0.009695,0.003770,0.022915,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.005783,0.0,0.0,0.006336,0.0,0.0,0.006032,0.0,0.0,0.005633,0.0,0.0,0.006204,6.894,1,1.5693,2.9830,3.5060,2.9683,3.4773
2,2,id_001f94081_2,A,3,G,A,.,.,.,E,E,E,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.027590,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.002879,0.000000,0.0,0.000000,0.017502,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,6.894,1,1.1227,0.2526,0.3008,0.2589,0.9988
3,3,id_001f94081_3,A,4,A,A,.,.,.,E,E,E,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.001278,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,6.894,1,0.8686,1.3789,1.0108,1.4552,1.3228
4,4,id_001f94081_4,A,5,A,A,.,.,(,E,E,S,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.002843,0.0,0.0,0.0,...,0.0,0.0,0.001408,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,6.894,1,0.7217,0.6376,0.2635,0.7244,0.7877


In [ ]:
data=data.drop(['Unnamed: 0','signal_to_noise','SN_filter'],axis=1)

In [ ]:
cols = ['id','sequence','seq_be','seq_af','structure',	'struct_be',	'struct_af',	'predicted_loop_type',	'loop_type_be',	'loop_type_af']

for col in cols:
    data[col] = data[col].astype('category')

In [ ]:
print (data.dtypes)

id                category
sequence          category
index_sequence       int64
seq_be            category
seq_af            category
                    ...   
reactivity         float64
deg_Mg_pH10        float64
deg_pH10           float64
deg_Mg_50C         float64
deg_50C            float64
Length: 123, dtype: object


In [ ]:
for i in range(107):
  cols.append('bpps_'+str(i))

In [ ]:
X = data[cols]
X.drop('id',axis=1)

Y = data[["reactivity",	"deg_Mg_pH10"	,"deg_pH10"	,"deg_Mg_50C",	"deg_50C"]]

In [ ]:
X.shape

(163200, 117)

# Modèle MLP (A continuer car BUG)

In [ ]:
from keras.models import Model,Sequential
from keras.layers import Input, Dense, BatchNormalization, Dropout, ReLU, Activation, Flatten,Conv2D, MaxPooling2D
from keras.backend import clear_session
clear_session()

In [ ]:
model_MLP = Sequential()


model_MLP.add(Dense(128,input_shape=(107,))) 
model_MLP.add(BatchNormalization())
model_MLP.add(ReLU())
model_MLP.add(Dropout(0.25))

model_MLP.add(Dense(64)) 
model_MLP.add(BatchNormalization())
model_MLP.add(ReLU())
model_MLP.add(Dropout(0.25))

model_MLP.add(Dense(5, activation=None))

model_MLP.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               13824     
_________________________________________________________________
batch_normalization (BatchNo (None, 128)               512       
_________________________________________________________________
re_lu (ReLU)                 (None, 128)               0         
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
batch_normalization_1 (Batch (None, 64)                256       
_________________________________________________________________
re_lu_1 (ReLU)               (None, 64)                0

In [ ]:
from keras.optimizers import Adam
opt = Adam(lr=0.001)

In [ ]:
model_MLP.compile(loss=mcrmse,
              optimizer=opt)

In [ ]:
%%time
batch_size = 64
epochs = 25
hist = model_MLP.fit(X, Y,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(X, Y),
              shuffle=True)

KeyboardInterrupt: ignored